In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path= "/content/drive/My Drive/finviz-3.csv"
df = pd.read_csv(path)

In [4]:
print(df.columns.tolist())

['Ticker', 'Company', 'Index', 'Sector', 'Industry', 'Country', 'Market Cap', 'P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'P/Cash', 'P/Free Cash Flow', 'Book/sh', 'Cash/sh', 'Dividend', 'Dividend Yield', 'Payout Ratio', 'EPS (ttm)', 'EPS next Q', 'EPS growth this year', 'EPS growth next year', 'EPS growth past 5 years', 'EPS growth next 5 years', 'Sales growth past 5 years', 'Sales growth quarter over quarter', 'EPS growth quarter over quarter', 'Sales', 'Income', 'EPS Surprise', 'Revenue Surprise', 'Shares Outstanding', 'Shares Float', 'Float %', 'Insider Ownership', 'Insider Transactions', 'Institutional Ownership', 'Institutional Transactions', 'Float Short', 'Short Ratio', 'Short Interest', 'Return on Assets', 'Return on Equity', 'Return on Investment', 'Current Ratio', 'Quick Ratio', 'LT Debt/Equity', 'Total Debt/Equity', 'Gross Margin', 'Operating Margin', 'Profit Margin', 'Beta', 'Average True Range', 'Volatility (Week)', 'Volatility (Month)', '20-Day Simple Moving Average', '50-

In [5]:
target = 'Price'
y = df[target]
X = df.drop(target, axis=1)

# Encode categorical features
categorical_features = X.dtypes == object
categorical_cols = X.columns[categorical_features]
label_encoder = LabelEncoder()
X[categorical_cols] = X[categorical_cols].apply(lambda col: label_encoder.fit_transform(col))

# One-hot encode categorical features
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
X_encoded = one_hot_encoder.fit_transform(X[categorical_cols])

# Combine one-hot encoded features with numeric features
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X = pd.concat([X[numeric_cols], pd.DataFrame(X_encoded.toarray())], axis=1)

# Impute missing values for numeric features
for feature in numeric_cols:
    imputer = SimpleImputer(strategy='mean')
    X[feature] = imputer.fit_transform(X[feature].values.reshape(-1, 1))


/usr/local/lib/python3.10/dist-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: [0]. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a random forest regression model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf.fit(X_train, y_train)

# Calculate the permutation feature importance
perm_importance = permutation_importance(rf, X_test, y_test)

# Sort the features by importance and display the results
sorted_idx = perm_importance.importances_mean.argsort()
feature_names = X_train.columns[sorted_idx]
importance_values = perm_importance.importances_mean[sorted_idx]

for name, value in zip(feature_names, importance_values):
    print(f"{name}: {value:.3f}")